<a href="https://colab.research.google.com/github/kassiaoliveiraa/palestracampuspartynordeste/blob/main/palestracampusparty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install pyarrow
!pip install pandas

In [31]:
from pyspark.sql import SparkSession
import pandas as pd
import os
import shutil


spark = SparkSession.builder.appName("ETL").getOrCreate()

In [ ]:
file_path = "/home/etl/processo_inicio_1990.csv"
file_size_bytes = os.path.getsize(file_path)
file_size_kb = file_size_bytes / 1024  # Converte de bytes para kilobytes

print(f"O tamanho do arquivo é {file_size_kb:.2f} KB")

In [ ]:
df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("sep", ";") \
    .load("/home/etl/processo_inicio_1990.csv")

df.show()

In [ ]:
df.createOrReplaceTempView("processos")

queryCount = """
SELECT count(*) AS count
FROM processos
"""

queryprocAno = """
SELECT count(*) AS count, DTABERTURA
FROM processos
GROUP BY DTABERTURA
"""

result = spark.sql(queryCount)

result.show()

In [41]:
df.write.parquet("/home/etl/processos_temp/")

In [ ]:
temp_folder = '/home/etl/processos_temp/'
destination_folder = '/home/etl/procesosparquet/'

os.makedirs(destination_folder, exist_ok=True)

file_counter = 1

for file_name in os.listdir(temp_folder):
    if file_name.endswith(".parquet"):
        source_file = os.path.join(temp_folder, file_name)

        destination_file = os.path.join(destination_folder, f"processos_{file_counter}.parquet")

        try:
            shutil.move(source_file, destination_file)
            file_counter += 1
        except FileNotFoundError as e:
            print(f"Erro ao mover o arquivo {source_file}: {e}")
        except Exception as e:
            print(f"Erro inesperado: {e}")

try:
    shutil.rmtree(temp_folder)
except FileNotFoundError as e:
    print(f"Erro ao remover a pasta temporária: {e}")

print("Todos os arquivos foram salvos com sucesso e renomeados.")

In [ ]:
df = spark.read \
    .format("parquet") \
    .load("/home/etl/procesosparquet")

df.count()

In [ ]:
def get_size_of_folder_in_kb(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.endswith(".parquet"):
                file_path = os.path.join(dirpath, filename)
                total_size += os.path.getsize(file_path)
    return total_size / 1024

folder_path = '/home/etl/procesosparquet'
total_size_kb = get_size_of_folder_in_kb(folder_path)

print(f"Tamanho total dos arquivos Parquet: {total_size_kb:.2f} KB")